# ClutchAI MVP

__GOAL__ 

Create a ReACT AI Agent for Yahoo Fantasy Basketball League.

__MVP__
1. Get live data from YahooFantasy API using 
2. Create vectorDB with documents from a LockedOn Basketball Podcast episode
3. Create a ReACT agent with both tools

__Test Cases__
- What is our league name?
- What are the key summaries from podcast for drafting?

#### 0. Environment Setup

In [8]:
import os
from dotenv import load_dotenv
from pathlib import Path
import json

# Load .env file from the project root (parent directory of notebook/)
# Resolve the path to get the absolute path to the .env file
env_file_location = Path('..').resolve()
load_dotenv(dotenv_path=env_file_location / ".env")
print(f"Loading .env from: {env_file_location / '.env'}\n")

# Set env variables
YAHOO_CONSUMER_KEY = os.environ.get('YAHOO_CLIENT_ID', "<INSERT_>")
YAHOO_CONSUMER_SECRET = os.environ.get('YAHOO_CLIENT_SECRET', "<INSERT>")
YAHOO_LEAGUE_ID = 58930
GAME_CODE = "nba"
GAME_ID=466

Loading .env from: /Users/matt/Code/ClutchAI/.env



In [9]:
# Show which env variables were loaded
env_vars_to_check = ['YAHOO_CLIENT_ID', 'YAHOO_CLIENT_SECRET', 'OPENAI_API_KEY', 'LANGSMITH_API_KEY']
print("Environment variables loaded:")
for var in env_vars_to_check:
    value = os.environ.get(var)
    if value:
        print(f"  ✓ {var}: {value[:10]} + ...")
    else:
        print(f"  ✗ {var}: NOT SET")

Environment variables loaded:
  ✓ YAHOO_CLIENT_ID: dj0yJmk9Ml + ...
  ✓ YAHOO_CLIENT_SECRET: 967c74034e + ...
  ✓ OPENAI_API_KEY: sk-proj-4A + ...
  ✓ LANGSMITH_API_KEY: lsv2_pt_7c + ...


#### 1. Connect to YahooFantasy League Data

In [10]:
from yfpy.query import YahooFantasySportsQuery

In [11]:
query = YahooFantasySportsQuery(
    league_id=YAHOO_LEAGUE_ID,
    game_code="nba",
    game_id= GAME_ID,
    env_var_fallback = True,
    env_file_location = env_file_location,
    save_token_data_to_env_file = True,
)

In [12]:
#Data to pull from Yahoo Fantasy
league_meta = query.get_league_metadata()
league_meta_dict = json.loads(league_meta.to_json())
print(len(league_meta))
league_meta_dict

31


{'allow_add_to_dl_extra_pos': 1,
 'current_week': 4,
 'draft_status': 'postdraft',
 'edit_key': '2025-11-14',
 'end_date': '2026-04-05',
 'end_week': 23,
 'felo_tier': 'gold',
 'game_code': 'nba',
 'iris_group_chat_id': None,
 'is_cash_league': 0,
 'is_highscore': False,
 'is_plus_league': 0,
 'is_pro_league': 0,
 'league_id': '58930',
 'league_key': '466.l.58930',
 'league_type': 'private',
 'league_update_timestamp': 1763110527,
 'logo_url': 'https://yahoofantasysports-res.cloudinary.com/image/upload/t_s192sq/fantasy-logos/56479288575_ec9899.jpg',
 'matchup_week': 4,
 'name': 'TK Fiji Fantasy 2024/25',
 'num_teams': 12,
 'renew': '454_14696',
 'renewed': None,
 'roster_type': 'date',
 'scoring_type': 'headone',
 'season': 2025,
 'start_date': '2025-10-21',
 'start_week': 1,
 'url': 'https://basketball.fantasysports.yahoo.com/nba/58930',
 'weekly_deadline': 'intraday'}

In [15]:
query.get_user_teams()

[Game({
   "code": "nba",
   "game_id": 322,
   "game_key": "322",
   "is_game_over": 1,
   "is_offseason": 1,
   "is_registration_over": 1,
   "name": "Basketball",
   "season": 2013,
   "teams": {
     "team": {
       "draft_position": 9,
       "has_draft_grade": 0,
       "is_owned_by_current_login": 1,
       "league_scoring_type": "head",
       "managers": {
         "manager": {
           "felo_score": 570,
           "felo_tier": "bronze",
           "guid": "Q2Z2T2CXBSK6JYVRUBREBQTHFI",
           "image_url": "https://s.yimg.com/ag/images/default_user_profile_pic_64sq.jpg",
           "is_current_login": 1,
           "manager_id": 11,
           "nickname": "Matthew"
         }
       },
       "name": "Matthew's Team",
       "number_of_moves": 0,
       "number_of_trades": 0,
       "roster_adds": {
         "coverage_type": "week",
         "coverage_value": 24,
         "value": 0
       },
       "team_id": 11,
       "team_key": "322.l.181324.t.11",
       "team_log

In [16]:
query.get_team_info('454.l.14696.t.6')

2025-11-14 15:42:18.791 - ERROR - query.py - yfpy.query:456 - Attempt to retrieve data at URL https://fantasysports.yahooapis.com/fantasy/v2/team/466.l.58930.t.454.l.14696.t.6;out=metadata,stats,standings,roster,draftresults,matchups?format=json failed with error: "Invalid team key 466.l.58930.t.454.l.14696.t.6 provided."


YahooFantasySportsDataNotFound: Attempt to retrieve data at URL https://fantasysports.yahooapis.com/fantasy/v2/team/466.l.58930.t.454.l.14696.t.6;out=metadata,stats,standings,roster,draftresults,matchups?format=json failed with error: "Invalid team key 466.l.58930.t.454.l.14696.t.6 provided."

In [17]:
query.get_team_info(6)

Team({
  "draft_results": [
    {
      "draft_result": {
        "pick": 8,
        "round": 1,
        "team_key": "466.l.58930.t.6",
        "player_key": "466.p.6703"
      }
    },
    {
      "draft_result": {
        "pick": 20,
        "round": 2,
        "team_key": "466.l.58930.t.6",
        "player_key": "466.p.5835"
      }
    },
    {
      "draft_result": {
        "pick": 32,
        "round": 3,
        "team_key": "466.l.58930.t.6",
        "player_key": "466.p.6021"
      }
    },
    {
      "draft_result": {
        "pick": 44,
        "round": 4,
        "team_key": "466.l.58930.t.6",
        "player_key": "466.p.6035"
      }
    },
    {
      "draft_result": {
        "pick": 56,
        "round": 5,
        "team_key": "466.l.58930.t.6",
        "player_key": "466.p.6707"
      }
    },
    {
      "draft_result": {
        "pick": 68,
        "round": 6,
        "team_key": "466.l.58930.t.6",
        "player_key": "466.p.6019"
      }
    },
    {
      "draft_

#### 2. Connecting LangChain Agent to Yahoo API

In [6]:
from langchain_core.tools import tool
from langchain.agents import create_agent
from langchain_openai import ChatOpenAI
from langchain_core.messages import AIMessage


In [9]:
#Creat YahooFantasy Agent
class YahooFantasyAgent:
    def __init__(self):
        self.YAHOO_CONSUMER_KEY = os.environ.get('YAHOO_CLIENT_ID', "<INSERT_>")
        self.YAHOO_CONSUMER_SECRET = os.environ.get('YAHOO_CLIENT_SECRET', "<INSERT>")
        self.YAHOO_LEAGUE_ID = 58930
        self.GAME_CODE = "nba"
        self.GAME_ID=466

        self.query = YahooFantasySportsQuery(
            league_id=YAHOO_LEAGUE_ID,
            game_code="nba",
            game_id= GAME_ID,
            yahoo_consumer_key = YAHOO_CONSUMER_KEY,
            yahoo_consumer_secret = YAHOO_CONSUMER_SECRET,
            env_var_fallback = True,
            env_file_location = env_file_location,
            save_token_data_to_env_file = True,
        )

    def get_league_metadata(self) -> str:
        """Fetch Yahoo Fantasy league metadata."""
        try:
            data = self.query.get_league_metadata()
            return f'The Yahoo Fantasy league metadata in json format is: {data}'
        except Exception as e:
            return f"Failed to Yahoo Fantasy league metadata: {e}"
        
@tool("YahooLeagueMetDataTool", description="Get Yahoo League Metadata in json format from YPFS.")
def league_metadata_tool() -> str:
    """Get Yahoo League Metadata."""
    return YahooFantasyAgent().get_league_metadata()

In [11]:
#Create Clutch AI Agent
tools = [league_metadata_tool]
llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)

agent_v1 = create_agent(
    model = llm,
    tools = tools,
    system_prompt="You are a helpful assistant for a Yahoo Fantasy Sports league manager.",
)

In [9]:
#Test Agent
inputs = {"messages": [{"role": "user", "content": "What is the Yahoo Fantasy League Name?"}]}
agent_v1.invoke(inputs)

{'messages': [HumanMessage(content='What is the Yahoo Fantasy League Name?', additional_kwargs={}, response_metadata={}, id='0732860d-244f-44b6-bb5b-990cde41b7fd'),
  AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 14, 'prompt_tokens': 67, 'total_tokens': 81, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_560af6e559', 'id': 'chatcmpl-CZ0IjtiHWHr1QUjJ9LQkNGMwmbpjw', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--18b03830-37ba-4409-9e68-ba2a49edd970-0', tool_calls=[{'name': 'YahooLeagueMetDataTool', 'args': {}, 'id': 'call_f7fLzHxSdQMlnYp2iy0Izota', 'type': 'tool_call'}], usage_metadata={'input_tokens': 67, 'output_tokens': 14, 'tot

In [10]:
#Stream Agent Response for Debug
for event in agent_v1.stream(inputs):
    print(event)

{'model': {'messages': [AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 14, 'prompt_tokens': 67, 'total_tokens': 81, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_560af6e559', 'id': 'chatcmpl-CZ0InZoXDaWA6cD3N7JF8JVvk7cxZ', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--cd9a3d43-8692-4d8c-b5bb-26d49cbdca8f-0', tool_calls=[{'name': 'YahooLeagueMetDataTool', 'args': {}, 'id': 'call_G0vPVyZnLHGxuGZIPo2clSPw', 'type': 'tool_call'}], usage_metadata={'input_tokens': 67, 'output_tokens': 14, 'total_tokens': 81, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})]}}
{'tools': {'mes

#### 2. Adding Youtube Transcripts to VectorDB

In [7]:
from langchain_community.vectorstores import Chroma
from langchain_community.document_loaders import YoutubeLoader
from langchain_community.document_loaders.youtube import TranscriptFormat
# from youtube_transcript_api import YouTubeTranscriptApi
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

In [8]:
# Get Documents from YouTube
loader = YoutubeLoader.from_youtube_url(
    "https://youtu.be/TB2QwCRMams?si=OQf8lK5esDB3BxuO", 
    add_video_info=False,
    transcript_format=TranscriptFormat.CHUNKS,
    chunk_size_seconds=30
)
docs = loader.load()
docs[0]

Document(metadata={'source': 'https://www.youtube.com/watch?v=TB2QwCRMams&t=0s', 'start_seconds': 0, 'start_timestamp': '00:00:00'}, page_content="There are a lot of players who are either hurt, injury-prone, susceptible to tanking. So, what do you do in a fantasy draft? When do you take them? Michael Bolton, he's going to give you some answers. >> Thanks, Josh. It's Michael Bolton here, and it's time for another episode of the Locked On Fantasy Basketball Podcast. Let's get to it. >> Let's get to it. Indeed. You are Locked on Fantasy, your daily")

In [ ]:
# Note: persist_directory is now defined in the cell where vectorstore is created (Cell 19)


In [15]:
import sys
sys.getsizeof(docs)

664

In [19]:
#Embedding YouTube Documents
# Note: In ChromaDB 0.4.x+, persistence is automatic when persist_directory is specified
persist_directory = "./mvp_chroma_db"
vectorstore = Chroma.from_documents(
    documents=docs, 
    embedding=OpenAIEmbeddings(),
    persist_directory=persist_directory  # Persistence happens automatically
)

retriever = vectorstore.as_retriever()

@tool("locked_on_retreiver", description="Retrieve contextual knowledge from Locked On Basketball.")
def retrieve_LockedOnKnowledge(query: str) -> str:
    """Retrieve contextual knowledge from Locked On Podcast YouTube transcripts or articles."""
    results = retriever.invoke(query)
    return "\n\n".join([r.page_content for r in results])

In [ ]:
# The vectorstore is already persisted to "./mvp_chroma_db" from the previous cell
import os

def get_dir_size(path):
    """Calculate total size of directory in bytes."""
    total = 0
    for dirpath, dirnames, filenames in os.walk(path):
        for filename in filenames:
            filepath = os.path.join(dirpath, filename)
            if os.path.exists(filepath):
                total += os.path.getsize(filepath)
    return total

print(f"Vectorstore persisted to: {persist_directory}")
print(f"Document count: {vectorstore._collection.count()}")
storage_size_bytes = get_dir_size(persist_directory)
storage_size_mb = storage_size_bytes / (1024 * 1024)
print(f"Storage size: {storage_size_mb:.2f} MB")

Vectorstore persisted to: ./mvp_chroma_db
Document count: 75
Storage size: 1.72 MB


In [14]:
#Create ReACT Agent with YouTube Knowledge Retrieval and Yahoo Fantasy Tool
#Note: Temperature is a parameter that controls the “creativity” or randomness of the text generated.

llm = ChatOpenAI(model="gpt-4o-mini")
tools = [league_metadata_tool, retrieve_LockedOnKnowledge]

agent_v2 = create_agent(
    model = llm,
    tools = tools,
    system_prompt="You are a helpful assistant for a Yahoo Fantasy Sports league manager.",
)

In [21]:
#Test Agent
inputs = {"messages": [{"role": "user", "content": "Give me 3 key draft advice from Locked On Podcast that I should follow for my league."}]}
output_v2 = agent_v2.invoke(inputs)

In [22]:
print(output_v2)

{'messages': [HumanMessage(content='Give me 3 key draft advice from Locked On Podcast that I should follow for my league. Provide sources.', additional_kwargs={}, response_metadata={}, id='d75e4a65-1991-40ca-9787-cd5948ca70c5'), AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 19, 'prompt_tokens': 106, 'total_tokens': 125, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_560af6e559', 'id': 'chatcmpl-CZ2mschdYzSy3zkJ20O9sVsFZdmg3', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--35ccf9ce-2f13-435b-ab6f-eeac204a8b74-0', tool_calls=[{'name': 'locked_on_retreiver', 'args': {'query': 'draft advice'}, 'id': 'call_3E8rKbt8Z0I1EEjjVlUgTJYN', 